In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
df=pd.read_csv("twitter2.csv")

In [ ]:
df = df.dropna()


In [ ]:
# Preprocessing steps
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    # Tokenize the tweet
    words = word_tokenize(tweet.lower())

    # Remove stopwords and lemmatize the words
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]

    # Join the words back into a string
    processed_tweet = ' '.join(words)

    return processed_tweet

# Apply preprocessing to the tweets
df['processed_tweets'] = df['clean_text'].apply(preprocess_tweet)


In [ ]:
X = df['processed_tweets']
y = df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test = vectorizer.transform(X_test)

In [ ]:
classifier = LogisticRegression()

# Train the classifier
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# Make predictions on the testing data
y_pred = classifier.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

        -1.0       0.87      0.76      0.81      7152
         0.0       0.85      0.95      0.90     11067
         1.0       0.91      0.89      0.90     14375

    accuracy                           0.88     32594
   macro avg       0.88      0.87      0.87     32594
weighted avg       0.88      0.88      0.88     32594



In [ ]:
def predict_sentiment(tweet):
    # Preprocess the tweet
    processed_tweet = preprocess_tweet(tweet)

    # Transform the preprocessed tweet using the TF-IDF vectorizer
    transformed_tweet = vectorizer.transform([processed_tweet])

    # Make the sentiment prediction
    prediction = classifier.predict(transformed_tweet)[0]

    # Map the prediction label to sentiment category
    sentiment = {-1.0: 'Negative', 0.0: 'Neutral', 1.0: 'Positive'}[prediction]

    # Get the probabilities for each class
    probabilities = classifier.predict_proba(transformed_tweet)

    # Get the index of the predicted sentiment category
    sentiment_index = np.where(classifier.classes_ == prediction)[0][0]

    # Get the probability for the predicted sentiment
    sentiment_probability = probabilities[0][sentiment_index]

    # Print the sentiment and its probability
    print(f"Sentiment: {sentiment}")
    print(f"Probability: {sentiment_probability:.4f}")

In [ ]:
tweet = "Get out of here"
predict_sentiment(tweet)

Sentiment: Neutral
Probability: 0.6177


In [ ]:
import streamlit as st

# Set page title
st.title("Twitter Sentiment Analysis")

# Create input text box for the user to enter a tweet
tweet_input = st.text_input("Enter a tweet:")

# Create a button to trigger sentiment prediction
if st.button("Predict Sentiment"):
    # Perform sentiment prediction
    predict_sentiment(tweet_input)


2023-07-06 22:18:08.317 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 45.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=8cc242649f1ef71f2a8727bf6314c00031527e495addaa8bf96f7a9872c52604
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzloc

In [ ]:
    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


SyntaxError: ignored

In [ ]:
 streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py

SyntaxError: ignored

In [ ]:
import pickle

# Save the trained model
with open('classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)


In [ ]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)